## SMIB

Synchronous Machine Infinite Bus (SMIB) system


### System building

In [1]:
from pydae.bmapu import bmapu_builder
import sympy as sym

#### System data

In [2]:
data = {
"system":{"name":"smib","S_base":100e6, "K_p_agc":0.0,"K_i_agc":0.0,"K_xif":0.01},       
"buses":[{"name":"1", "P_W":0.0,"Q_var":0.0,"U_kV":20.0},
         {"name":"2", "P_W":0.0,"Q_var":0.0,"U_kV":20.0}
        ],
"lines":[{"bus_j":"1", "bus_k":"2", "X_pu":0.05,"R_pu":0.01,"Bs_pu":1e-6,"S_mva":100.0}],
"syns":[
      {"bus":"1","S_n":100e6,
         "X_d":1.8,"X1d":0.3, "T1d0":8.0,    
         "X_q":1.7,"X1q":0.55,"T1q0":0.4,  
         "R_a":0.01,"X_l": 0.2, 
         "H":5.0,"D":1.0,
         "Omega_b":314.1592653589793,"omega_s":1.0,"K_sec":0.0,
         "avr":{"type":"sexs","K_a":100.0,"T_a":0.1,"T_b":1.0,"T_e":0.1,"E_min":-10.0,"E_max":10.0,"v_ref":1.0},
         "K_delta":0.0}],
"genapes":[{"bus":"2","S_n":1e9,"F_n":50.0,"X_v":0.001,"R_v":0.0,"K_delta":0.001,"K_alpha":1e-6}]
}

#### System generation and compilation

In [3]:
grid = bmapu_builder.bmapu(data)
grid.checker()
grid.uz_jacs = True

In [4]:
grid.construct()

# add power injections to bus '1', in pu in system base
p_1,q_1 = sym.symbols('p_1,q_1', real=True)
idx_bus = grid.buses_list.index('1')
grid.dae['g'][idx_bus*2]   += -p_1
grid.dae['g'][idx_bus*2+1] += -q_1
grid.dae['u_ini_dict'].update({'p_1':0.0,'q_1':0.0})
grid.dae['u_run_dict'].update({'p_1':0.0,'q_1':0.0})
grid.dae['h_dict'].update({'p_1':p_1,'q_1':q_1})

In [5]:
grid.compile('smib')